In [2]:
import os
import json
import gradio as gr
from dotenv import load_dotenv
from google import genai

In [3]:
load_dotenv(override=True)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
MODEL = "gemini-2.5-flash"

client = genai.Client(api_key=GEMINI_API_KEY)

In [4]:
system_message = """
You are an AI assistant for an airline company called Flightly. 
Give short, courteous answers, no more than one sentence. 
Always be polite and professional in your responses.
You only know about airline data. If you don't know the answer, say so."""

In [15]:
def chat(message, history):
    google_messages = []

    google_messages.append({"role": "user", "parts": [{"text": system_message}]})
    google_messages.append({"role": "model", "parts": [{"text": "Entendido, seguiré esas instrucciones."}]})

    for h in history:
        role = "user" if h["role"] == "user" else "model"
        google_messages.append({
            "role": role,
            "parts": [{"text": h["content"]}]
        })
    google_messages.append({"role": "user", "parts": [{"text": message}]})
    response = client.models.generate_content(
        model=MODEL,
        contents=google_messages
    )
    
    return response.text

gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [7]:
ticket_prices = {"london": "$800", "paris": "$900", "tokyo": "$1400", "berlin": "$500"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknow ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

In [8]:
get_ticket_price("London")

Tool called for city London


'The price of a ticket to London is $800'

In [9]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalPropperties": False
    }
}

In [10]:
tools = [{"type": "function", "function": price_function}]

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalPropperties': False}}}]

In [17]:
def handle_tool_call(message):
    tool_call = message.toll_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [18]:
def chat(message, history):
    gemini_history = []
    for h in history:
        role = "user" if h["role"] == "user" else "model"
        gemini_history.append({"role": role, "parts": [h["content"]]})
    chat_session = client.models.start_chat(history=gemini_history)
    response = chat_session.send_message(message)
    return response.text

In [ ]:
gr.ChatInterface(fn=chat).launch()

TypeError: ChatInterface.__init__() got an unexpected keyword argument 'type'